In [22]:
import pandas as pd
x = pd.read_csv("trains.csv")
from copy import copy
import warnings
warnings.filterwarnings("ignore")

In [2]:
def find(data, col, how, what):
    if how == "==": return data.iloc[(data[col]==what).index[data[col]==what]]
    elif how == ">": return data.iloc[(data[col]>what).index[data[col]==what]]
    elif how == ">=": return data.iloc[(data[col]>what).index[data[col]==what]]
    elif how == "<": return data.iloc[(data[col]<what).index[data[col]==what]]
    elif how == "<=": return data.iloc[(data[col]<=what).index[data[col]==what]]

def diff(want, rem):
    w = copy(want)
    for j in w:
        if j in rem: w.remove(j)
    for j in rem:
        if j in w: w.remove(j)
    return w

def exc(x, l):
    if type(l) == str: l = [l]
    return x[diff(x.columns.tolist(),l)]

In [3]:
y = copy(x)
y["Occupancy Status"] = y["Occupancy Status"].astype("category")
y["Occupancy Status"] = y["Occupancy Status"].replace({'STANDING_ROOM_ONLY':3,
                                                       'FEW_SEATS_AVAILABLE':2,
                                                      'MANY_SEATS_AVAILABLE':1})

y["Actual.Station.Dprt.Time"] = y["Actual.Station.Dprt.Time"].astype("datetime64[ns]")
y["Actual.Station.Arrv.Time"] = y["Actual.Station.Arrv.Time"].astype("datetime64[ns]")

y["Start_Time"] = y["Actual.Station.Arrv.Time"].dt.hour
y["Start_Minutes"] = y["Actual.Station.Arrv.Time"].dt.minute

y["Hour"] = (y["Actual.Station.Arrv.Time"].dt.minute>=30)*1 + y["Actual.Station.Arrv.Time"].dt.hour

train = y.groupby(by = ["day","Hour","Actual.Stop.Station"]).agg({'Occupancy Status':'max','Start_Time':'mean',\
                                                                 'Start_Minutes':'mean'}).reset_index()
train["Actual.Stop.Station"] = train["Actual.Stop.Station"].str.lower()
train["suburb"] = train["Actual.Stop.Station"]

In [4]:
postcodes = pd.read_csv("postcodes.csv")
postcodes["suburb"] = postcodes["suburb"].str.lower()
postcodes = postcodes.iloc[(postcodes["state"]=='NSW').index[(postcodes["state"]=='NSW')]]
postcodes = postcodes[["postcode","suburb","lat","lon"]]

In [5]:
data = train.merge(postcodes, left_on = "suburb", right_on = "suburb", how = "left")
data.columns = ["day","hour","train_stop","train_crowding","time","journey_time","suburb","postcode","lat","long"]
data = exc(data, ["suburb","seq","hour"])

In [6]:
bus = pd.read_csv("bus_capacity_clean.csv")
bus["capacity"] = bus["capacity"].replace({'STANDING_ROOM_ONLY':3,
                                                       'FEW_SEATS_AVAILABLE':2,
                                                      'MANY_SEATS_AVAILABLE':1})
bus["suburb"] = bus["suburb"].str.lower()
bus = exc(bus, "Unnamed: 0")
bus.columns = ["time","bus_stop","lat","long","bus_crowding","day"]

In [23]:
longs = {"wynyard":151.205547, "central":151.206667, "town hall":151.204758, "circular quay":151.212056, "martin place": 151.210358,
        "museum":151.20963}
lats = {"wynyard":-33.865868, "central":-33.8825, "town hall":33.873682, "circular quay":-33.860705, "martin place":-33.867733,
       "museum":-33.87575}

data["change"] = data["train_stop"]

for j in longs.keys():
    tochange = find(data, "train_stop", "==", j)
    tochange["lat"] = tochange["lat"].fillna(lats[j])
    tochange["long"] = tochange["long"].fillna(longs[j])
    tochange["postcode"] = tochange["postcode"].fillna(2000)
    tochange["change"] = "sydney"
    data.iloc[tochange.index] = tochange

In [8]:
data["time"] = data["time"].astype('float')
data["time"] = round(data["time"])
data["time"] = data["time"].astype('int')

In [9]:
cleaned = data.merge(bus, left_on = ["day","time","change"], right_on = ["day","time","bus_stop"])\
            .groupby(by = ["day","time","train_stop"]).agg({'train_crowding':'max','journey_time':'mean',
                                                           'postcode':'max','lat_x':'median',
                                                            'long_x':'median',
                                                           'bus_crowding':'max'}).reset_index()
cleaned = cleaned.dropna(subset=["postcode"])
cleaned.columns = ['day', 'time', 'train_stop', 'train_crowding', 'journey_time','postcode', 'lat', 'long', 'bus_crowding']
cleaned["postcode"] = cleaned["postcode"].astype(int)

In [10]:
ato = pd.read_excel("ato.xlsx",sheetname = "Data")
ato = ato[["Postcode","Individuals1","Net capital gain","Total income or loss","0-4 years","5-9 years",
"10-14 years","15-19 years","20-24 years","25-29 years","30-34 years","35-39 years","40-44 years","45-49 years","50-54 years","55-59 years","60-64 years",
"65-69 years","70-74 years","75-79 years","80-84 years","85-89 years","90-94 years","95-99 years","100 years and over","Oceanian","North-West European",
"Southern and Eastern European","North African and Middle Eastern","South-East Asian","North-East Asian","Southern and Central Asian",
"People of the Americas","Sub-Saharan African","Male","Female","under or 0","$1 to 1000","1000-1999","2000+","Married in a registered marriage",
"Married in a de facto marriage","Not married"]]
newcols = ["postcode","population","capital_gain","total_income","0-4_years","5-9_years",
"10-14_years","15-19_years","20-24_years","25-29_years","30-34_years","35-39_years","40-44_years","45-49_years","50-54_years","55-59_years","60-64_years",
"65-69_years","70-74_years","75-79_years","80-84_years","85-89_years","90-94_years","95-99_years","100_years+","Oceanian","North-West_European",
"Southern_Eastern_European","North_African_Middle_Eastern","South-East_Asian","North-East_Asian","Southern_Central_Asian",
"Americas","Sub-Saharan_African","m","f","$<=0","$1-1000","$1000-1999","$2000+","Married",
"defacto","single"]
newcols = [x.lower() for x in newcols]
ato.columns = newcols

In [11]:
aggs = {'train_crowding':'max','journey_time':'mean','lat':'median','long':'median','bus_crowding':'max'}
for k in newcols: aggs[k]='max'

In [12]:
cleaned = cleaned.merge(ato, on="postcode").groupby(by=["day","time","train_stop"]).agg(aggs).reset_index()

In [13]:
aq = pd.read_csv("aq.csv", delimiter =";")
aq = aq.iloc[1:]
aq.columns = (aq.T.reset_index()[1]).tolist()
aq = aq.iloc[1:].reset_index()
aq["Date"]=aq["Date"].str.split("/").str[0].str.lstrip("0")
aq["Time"]=(aq["Time"].str.split(":").str[0].str.lstrip("0")).astype("int")-1

aq.columns=pd.Series(aq.columns.tolist()).str.rstrip("AQI hourly AQI [index]").str.lower().tolist()

aqcols = ['', 'dat', 'tim', 'randwick', 'rozelle', 'lindfield', 'liverpool',
       'bringelly', 'chullor', 'earlwood', 'wyong', 'wallsend', 'carrington',
       'stockton', 'newcastle', 'mayfield', 'beresfield', 'tamworth',
       'wollongong', 'kembla grange', 'richmond', 'bargo', 'albury',
       'wagga wagga', 'st marys', 'vineyard', 'bathurst', 'macarthur',
       'oakdale', 'albion park', 'prospect', 'muswellbrook', 'singleton',
       'maison dieu', 'camberwell', 'singleton', 'mount thorley', 'bulg',
       'wagga wagga', 'muswellbrook', 'wybong', 'aberdeen',
       'singleton', 'jerrys plains', 'warkworth', 'merriw',
       'campbelltown', 'camden']
aq.columns = aqcols
aq = aq.melt(id_vars=['dat','tim'])
aq.columns = ["day","time","suburb","AQI"]
aq = aq.iloc[(aq["suburb"]!="").index[aq["suburb"]!=""]].reset_index()
aq = aq[["day","time","suburb","AQI"]]
aqposts = pd.DataFrame(aqcols).merge(postcodes, left_on=[0], right_on=["suburb"]).groupby(by="suburb").agg({'postcode':'max'}).reset_index()

In [14]:
sorts = aqposts.sort_values(by="postcode").reset_index()
df = {}
for j in range(sorts.shape[0]):
    if j == 0:
        for increments in range(sorts.iloc[j]["postcode"]):
            df[sorts.iloc[j]["postcode"]-increments] = sorts.iloc[j]["suburb"]
    elif j+1 < sorts.shape[0]:
        for increments in range(sorts.iloc[j+1]["postcode"]-sorts.iloc[j]["postcode"]):
            df[increments+sorts.iloc[j]["postcode"]] = sorts.iloc[j]["suburb"]
    else:
        for increments in range(2787-sorts.iloc[j]["postcode"]):
            df[sorts.iloc[j]["postcode"]+increments] = sorts.iloc[j]["suburb"]
            
filled = pd.DataFrame.from_dict(df,orient='index').reset_index()
filled.columns = ["postcode","suburb"]

aq=aq.reset_index()
aq=aq.iloc[(aq["AQI"]!='--').index[aq["AQI"]!='--'].tolist()]
aq = exc(aq, "index")

aq = aq.dropna(subset=["AQI"])
aq["AQI"] = aq["AQI"].astype("int")
aq_all = filled.merge(aq, on="suburb",how="left").groupby(by=["day","time","postcode"]).agg({"AQI":'max'})

In [17]:
aq_all=aq_all.reset_index()
aq_all["day"]=aq_all["day"].astype(int)

In [18]:
cleaned = cleaned.merge(aq_all, on = ["day","time","postcode"], how = "left")
cleaned = exc(cleaned, "index")

In [20]:
cleaned.to_csv("data.csv")